In [1]:
import numpy as np
import torch
import tqdm
from tqdm import tqdm
import torch.nn as nn
from torchsummary import summary
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn.functional as functional

# from google.colab import drive
# drive.mount('/content/gdrive')
PATH = "./Checkpoint/model.pt"

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'mps'
torch.backends.cudnn.benchmark = True
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()
%load_ext tensorboard

/Users/alimollahoseini/Github/VGG/vgg.venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
class VGG_Net(nn.Module):
  def __init__(self, num_classes, net_type="VGG-11", in_channels=3):
    super(VGG_Net, self).__init__()

    self.VGG = dict()
    self.VGG["VGG-11"] = [1, 1, 2, 2, 2] 
    self.VGG["VGG-13"] = [2, 2, 2, 2, 2] 
    self.VGG["VGG-16"] = [2, 2, 3, 3, 3] 
    self.VGG["VGG-19"] = [2, 2, 4, 4, 4]
    
    # Adding convolutional layers
    layers = []
    out_channels = 64
    for layer_numbers in self.VGG[net_type]:
      for _ in range(layer_numbers):
        layers += [nn.Conv2d(in_channels=in_channels, out_channels=out_channels, 
                             kernel_size=(3, 3), stride=1, padding=1),  nn.BatchNorm2d(out_channels), nn.ReLU()]
        in_channels=out_channels
      layers += [nn.MaxPool2d(kernel_size=(2, 2), stride=2)]
      if(out_channels != 512):
        out_channels *= 2
    self.conv_layers = nn.Sequential(*layers)

    # Adding fc layers
    self.fc_layers = nn.Sequential(
                        nn.Linear(512, 4096),
                        nn.ReLU(),
                        nn.Dropout(p=0.5),
                        nn.Linear(4096, 4096),
                        nn.ReLU(),
                        nn.Dropout(p=0.5),
                        nn.Linear(4096, num_classes))
    

  def forward(self, input):
    x = self.conv_layers(input)
    x = x.reshape(x.shape[0], -1)
    x = self.fc_layers(x)
    x = functional.softmax(x, dim=1)
    return x

In [3]:
def data_loader(
  data_dir,
  batch_size,
  random_seed=42,
  valid_size=0.1,
  shuffle=True,
  test=False):

  transformation = transforms.Compose([
              transforms.ToTensor(),
              transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
      ])

  if test:
        dataset = datasets.CIFAR100(
          root=data_dir, train=False,
        )

        data_loader = torch.utils.data.DataLoader(
            dataset, batch_size=batch_size, shuffle=shuffle
        )

        return data_loader

  train_dataset = datasets.CIFAR100(
      root=data_dir, train=True,
      download=True, transform=transformation,
  )
  valid_dataset = datasets.CIFAR10(
      root=data_dir, train=True,
      download=True, transform=transformation,
  )

  num_train = len(train_dataset)
  indices = list(range(num_train))
  split = int(np.floor(valid_size * num_train))

  if shuffle:
    np.random.seed(random_seed)
    np.random.shuffle(indices)

  train_idx, valid_idx = indices[split:], indices[:split]
  train_sampler = SubsetRandomSampler(train_idx)
  valid_sampler = SubsetRandomSampler(valid_idx)

  train_loader = torch.utils.data.DataLoader(
		train_dataset, batch_size=batch_size, sampler=train_sampler)

  valid_loader = torch.utils.data.DataLoader(
		valid_dataset, batch_size=batch_size, sampler=valid_sampler)

  return (train_loader, valid_loader)

In [5]:
train_loader, valid_loader = data_loader(data_dir='./data',
                                         batch_size=64)

test_loader = data_loader(data_dir='./data',
                              batch_size=64,
                              test=True)

Files already downloaded and verified
Files already downloaded and verified


In [9]:
num_classes = 100
num_epochs = 20
batch_size = 16
learning_rate = 0.001

model = VGG_Net(num_classes).to(device)


# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

# Training

In [13]:
total_step = len(train_loader)


for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(tqdm(train_loader)):  
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        one_hot_labels = functional.one_hot(labels, num_classes=100).float()
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, one_hot_labels)
        # torch.save({
        #     'epoch': epoch,
        #     'model_state_dict': model.state_dict(),
        #     'optimizer_state_dict': optimizer.state_dict(),
        #     'loss': loss,
        #     }, PATH)
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if i % 100 == 0:
          # Write into tensorboard
          writer.add_scalar("Loss/epoch/batch", loss, epoch, i)


    print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
    

    # Save a checkpoint
    # torch.save({
    #         'epoch': epoch,
    #         'model_state_dict': model.state_dict(),
    #         'optimizer_state_dict': optimizer.state_dict(),
    #         'loss': loss,
    #         }, PATH)


    # Validation
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in valid_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        del images, labels, outputs

    print('Accuracy of the network on the {} validation images: {} %'.format(5000, 100 * correct / total)) 
# writer.flush()


100%|██████████| 704/704 [01:57<00:00,  5.98it/s]


Epoch [1/20], Step [704/704], Loss: 4.6222


100%|██████████| 704/704 [01:53<00:00,  6.18it/s]


Epoch [2/20], Step [704/704], Loss: 4.6181


100%|██████████| 704/704 [01:55<00:00,  6.08it/s]


Epoch [3/20], Step [704/704], Loss: 4.6194


 22%|██▏       | 158/704 [00:26<01:31,  5.95it/s]


KeyboardInterrupt: 